In [1]:
 !pip install transformers

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [5]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [6]:
pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 56.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) 

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [8]:
import ktrain
from ktrain import text

In [9]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Token'].tolist()
y_train=train['Label'].tolist()
x_validation=validation['Token'].tolist()
y_validation=validation['Label'].tolist()
x_test=test['Token'].tolist()
y_test=test['Sentiment'].tolist()

In [10]:
x_train= np.array(x_train)
x_validation= np.array(x_validation)
x_test= np.array(x_test)


In [11]:
categories = ['angry','disgust','fear','happy','sad','surpise']

In [12]:
model_name = 'monsoon-nlp/bangla-electra'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

In [13]:
train = trans.preprocess_train(x_train,y_train)
valid = trans.preprocess_test(x_validation,y_validation)

preprocessing train...
language: bn
train sequence lengths:
	mean : 26
	95percentile : 74
	99percentile : 173


Downloading:   0%|          | 0.00/161 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 26
	95percentile : 76
	99percentile : 165


In [14]:
model = trans.get_classifier()

Downloading:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

In [15]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [16]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
weight={i:class_weight[i] for i in range(6)}

In [17]:
learner.fit_onecycle(2e-5,20,class_weight=weight) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/20
427/427 [==============================] - 45s 61ms/step - loss: 1.7915 - accuracy: 0.2342 - val_loss: 1.7877 - val_accuracy: 0.2137
Epoch 2/20
427/427 [==============================] - 22s 52ms/step - loss: 1.7911 - accuracy: 0.2100 - val_loss: 1.7870 - val_accuracy: 0.2777
Epoch 3/20
427/427 [==============================] - 22s 51ms/step - loss: 1.7867 - accuracy: 0.2303 - val_loss: 1.7861 - val_accuracy: 0.1607
Epoch 4/20
427/427 [==============================] - 23s 53ms/step - loss: 1.7812 - accuracy: 0.1784 - val_loss: 1.7904 - val_accuracy: 0.1591
Epoch 5/20
427/427 [==============================] - 23s 53ms/step - loss: 1.7749 - accuracy: 0.1690 - val_loss: 1.7734 - val_accuracy: 0.1716
Epoch 6/20
427/427 [==============================] - 22s 52ms/step - loss: 1.7613 - accuracy: 0.2063 - val_loss: 1.7279 - val_accuracy: 0.2418
Epoch 7/20
427/427 [==============================] - 22s 52ms/step - los

In [18]:
learner.validate(class_names=categories)

              precision    recall  f1-score   support

       angry       0.30      0.35      0.32       137
     disgust       0.25      0.04      0.07        72
        fear       0.08      0.40      0.14        30
       happy       0.65      0.61      0.63       195
         sad       0.29      0.04      0.07       132
     surpise       0.24      0.41      0.31        75

    accuracy                           0.34       641
   macro avg       0.30      0.31      0.26       641
weighted avg       0.38      0.34      0.32       641



array([[ 48,   4,  37,  15,   5,  28],
       [ 20,   3,  18,   9,   2,  20],
       [  5,   1,  12,   5,   1,   6],
       [ 33,   1,  23, 118,   3,  17],
       [ 38,   3,  36,  24,   5,  26],
       [ 15,   0,  18,  10,   1,  31]])

In [19]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [20]:
y_pred = predictor.predict(x_test)
print(y_pred)

['angry', 'fear', 'angry', 'surpise', 'angry', 'surpise', 'surpise', 'happy', 'angry', 'angry', 'fear', 'angry', 'happy', 'fear', 'fear', 'angry', 'fear', 'angry', 'fear', 'happy', 'angry', 'surpise', 'angry', 'happy', 'fear', 'happy', 'happy', 'angry', 'happy', 'fear', 'surpise', 'fear', 'happy', 'happy', 'angry', 'happy', 'angry', 'happy', 'happy', 'happy', 'sad', 'happy', 'fear', 'happy', 'happy', 'angry', 'fear', 'fear', 'surpise', 'surpise', 'fear', 'fear', 'happy', 'happy', 'angry', 'happy', 'surpise', 'surpise', 'happy', 'happy', 'fear', 'happy', 'angry', 'happy', 'angry', 'surpise', 'happy', 'fear', 'happy', 'fear', 'happy', 'fear', 'fear', 'happy', 'sad', 'fear', 'surpise', 'angry', 'fear', 'angry', 'surpise', 'surpise', 'happy', 'sad', 'surpise', 'happy', 'happy', 'happy', 'surpise', 'happy', 'angry', 'surpise', 'happy', 'happy', 'angry', 'fear', 'surpise', 'angry', 'fear', 'angry', 'happy', 'angry', 'fear', 'surpise', 'happy', 'happy', 'happy', 'surpise', 'surpise', 'angry',

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.37      0.37      0.37       144
     disgust       0.00      0.00      0.00        66
        fear       0.07      0.28      0.11        32
       happy       0.59      0.64      0.61       187
         sad       0.18      0.03      0.05       148
     surpise       0.18      0.38      0.25        63

    accuracy                           0.33       640
   macro avg       0.23      0.28      0.23       640
weighted avg       0.32      0.33      0.30       640

